In [18]:
import sys
sys.path.insert(0, "../")
import BatchPaSR as bp
from CanteraTools import *    
import plotly
import plotly.plotly as pplotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='edwin217', api_key='hoh1lSzTIVSeckYhrNYL')

### phi_global = 0.635

### phi_main = 0.3719 

### tau_main = 20 - 0.28 ms

### P = 25 kg

### total_mass = 1 kg

In [2]:
phi_global = 0.635
phi_main = 0.3719
tau_main = 20 - 0.28
tau_global = 20
tau_mix = [0.005, 0.01, 0.05, 0.1, 0.3]
P = 25 * ct.one_atm
miliseconds = 0.001
total_mass = 1
fs = 0.058387057492574147
# Go ahead and run main burner since everything is fixed
[mfm, mam, mfs, mas] = solvePhi_airSplit(phi_global, phi_main, total_mass, 1)
# print(f"mfm = {mfm:.3f}; mam = {mam:.3f}; mfs = {mfs:.3f}; mas = {mas:.3f}")
# print(f"phi_main = {mfm/mam/fs:.3f}; phi_sec = {mfs/mas/fs:.3f}; phi_global = {(mfm+mfs)/(mas+mam)/fs}")
[vit_reactor, main_burner_DF] = runMainBurner(phi_main, tau_main*milliseconds)
vit_particle = bp.Particle.fromGas(vit_reactor.thermo, particle_mass = mfm+mam)
fuel = ct.Solution('gri30.xml')
fuel.TPX = 300, P, {'CH4':1} # TODO: Come up with a more general way of doing secondary gas so that we can have both fuel and air 
air = ct.Solution('gri30.xml'); 
air.TPX = 650, P, {'O2':0.21, 'N2':0.79}
secondary_gas = mix([fuel, air], [mfs, mas], P = P)
secondary_part = bp.Particle.fromGas(secondary_gas, particle_mass=mfs+mas)

Vitiator end time: 19.719906328977743 milliseconds
Initial mainBurnerDF length: 379
New mainBurnerDF length: 1291


# Test 1: Running a PaSBR with Fixed Entrainment of Both Streams

In [3]:
mass_main = mfm + mam 
mass_sec = mfs + mas 
J_ratio = 3
rho_crossflow = vit_reactor.thermo.density
u_crossflow = 20 # m/s
rho_jet = secondary_gas.density
u_jet = ((J_ratio * (rho_crossflow * u_crossflow * u_crossflow))/rho_jet)**0.5
print(f"u_crossflow = {u_crossflow:.2f} m/s")
print(f"u_jet = {u_jet:.2f} m/s\n")
A_crossflow = 1 # m2
A_jet = A_crossflow/10

mdot_main = vit_reactor.thermo.density * A_crossflow * u_crossflow 
mdot_sec = rho_jet * A_jet * u_jet
print(f"Vitiated product mass = {mass_main:.3f} kg\nSecondary fluid mass = {mass_sec:.3f} kg\n")
print(f"Vitiated product density = {vit_reactor.thermo.density:.3f} kg/m3\nSecondary fluid density = {secondary_gas.density:.3f} kg/m3\n")
print(f"mdot_main = {mdot_main:.3f} kg/s\nmdot_sec = {mdot_sec:.3f} kg/s\n")
print(f"Time taken to completely entrain main burner fluid = {mass_main/mdot_main*1e3:.4f} ms")
print(f"Time taken to completely entrain jet fluid = {mass_sec/mdot_sec*1e3:.4f} ms")

u_crossflow = 20.00 m/s
u_jet = 20.61 m/s

Vitiated product mass = 0.985 kg
Secondary fluid mass = 0.015 kg

Vitiated product density = 5.765 kg/m3
Secondary fluid density = 16.292 kg/m3

mdot_main = 115.306 kg/s
mdot_sec = 33.573 kg/s

Time taken to completely entrain main burner fluid = 8.5441 ms
Time taken to completely entrain jet fluid = 0.4412 ms


In [4]:
pasbr = bp.PaSBR([], N_MAX=200, dt=0.001e-3)

In [5]:
t1 = time.time()
def massflowfcnmain(time):
    if time <= mass_main/mdot_main:
        return mdot_main
    else:
        return 0

def massflowfcnsec(time):
    if time <= mass_sec/mdot_sec:
        return mdot_sec
    else:
        return 0
totalTime = 10*milliseconds

dt_entrain = (mass_main/mdot_main)/30
print(f"dt_entrain = {dt_entrain/1e-3:.4f} ms")
t = np.arange(0, totalTime, pasbr.dt)
pfc_vit = bp.ParticleFlowController(pasbr, vit_reactor.thermo, mass_main, dt_entrain, method=massflowfcnmain)
# pfc_vit.gas()
pfc_sec = bp.ParticleFlowController(pasbr, secondary_gas, mass_sec, dt_entrain, method=massflowfcnsec)
# pfc_sec.gas()
for i in range(0,t.size):
    pfc_vit.entrain(t[i])
    pfc_sec.entrain(t[i])
#     pdb.set_trace()
    pasbr.react()
    pasbr.mix(tau_mix=0.001*milliseconds)
t2 = time.time(); 
print(f"Time taken = {(t2-t1)/60:.2f} minutes")

dt_entrain = 0.2848 ms
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.01 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.01 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg into PaSBR
Entraining particle of mass 0.03 kg i

In [39]:
df = pasbr.get_timeHistory()
particle_historyList = [p.get_timeHistory(dataFrame=True) for p in pasbr.particle_list]
for i in range(0, len(particle_historyList)):
    particle_historyList[i].age += totalTime - particle_historyList[i].age.iloc[-1]

In [49]:
T_traces = [go.Scatter(x = particle_historyList[i]['age'], y = particle_historyList[i]['T'], mode = 'lines', yaxis='y2', name = f"Particle {i} Temp") for i in range(0, len(particle_historyList))]

In [45]:
Y_NO_trace = go.Scatter(x = df['age'], y = df['Y_NO']*1e6, mode = 'lines', name = 'Y_NO Uncorrected'); 
X_NO_trace = go.Scatter(x = df['age'], y = df['X_NO']*1e6, mode = 'lines', name = 'X_NO Uncorrected'); 
X_NO_corr_trace = go.Scatter(x = df['age'], y = correctNOx(df['X_NO'], df['X_H2O'], df['X_O2']), name  = 'X_NO Corrected')

NO_and_T_traces = T_traces + [Y_NO_trace,  X_NO_trace,  X_NO_corr_trace]

layout = go.Layout(title='NO and Temp', yaxis={'title': "NO (ppm)"}, yaxis2={'title':"Temperature(K)", 'side':"right", 'overlaying':"y"})
NO_fig = go.Figure(data = NO_and_T_Traces, layout=multiAxisLayout)

pplotly.iplot(NO_and_T_traces, filename = 'NO_and_temp')

The draw time for this plot will be slow for all clients.


C:\Users\edwin\AppData\Local\Continuum\anaconda3\envs\canteraEnv\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [55]:
Y_NO_trace = go.Scatter(x = df['age'], y = df['Y_NO']*1e6, mode = 'lines', name = 'Y_NO Uncorrected'); 
X_NO_trace = go.Scatter(x = df['age'], y = df['X_NO']*1e6, mode = 'lines', name = 'X_NO Uncorrected'); 
X_NO_corr_trace = go.Scatter(x = df['age'], y = correctNOx(df['X_NO'], df['X_H2O'], df['X_O2']), name  = 'X_NO Corrected')

NO_and_T_traces = T_traces + [Y_NO_trace,  X_NO_trace,  X_NO_corr_trace]

multiAxisLayout = go.Layout(title='NO and Temp', yaxis={'title': "NO (ppm)"}, yaxis2={'title':"Temperature(K)", 'side':"right", 'overlaying':"y"})
NO_fig = go.Figure(data = NO_and_T_traces, layout=multiAxisLayout)

plot_url = pplotly.iplot(NO_fig, filename = 'NO_and_temp')
plot_url

The draw time for this plot will be slow for all clients.


C:\Users\edwin\AppData\Local\Continuum\anaconda3\envs\canteraEnv\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [57]:
Y_CO_trace = go.Scatter(x = df['age'], y = df['Y_CO']*1e6, mode = 'lines', name = 'Y_CO Uncorrected'); 
X_CO_trace = go.Scatter(x = df['age'], y = df['X_CO']*1e6, mode = 'lines', name = 'X_CO Uncorrected'); 
X_CO_corr_trace = go.Scatter(x = df['age'], y = correctNOx(df['X_CO'], df['X_H2O'], df['X_O2']), name  = 'X_CO Corrected')

CO_and_T_traces = T_traces + [Y_CO_trace,  X_CO_trace,  X_CO_corr_trace]

multiAxisLayout = go.Layout(title='CO and Temp', yaxis={'title': "CO (ppm)", 'type':'log'}, yaxis2={'title':"Temperature(K)", 'side':"right", 'overlaying':"y"})
CO_fig = go.Figure(data = CO_and_T_traces, layout=multiAxisLayout)

CO_url = pplotly.iplot(CO_fig, filename = 'CO_and_temp')
CO_url

The draw time for this plot will be slow for all clients.


C:\Users\edwin\AppData\Local\Continuum\anaconda3\envs\canteraEnv\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Too Long

